In [1]:
from embedding_generator import EmbeddingGenerator
import torch
import numpy as np
import json
from typing import List, Tuple

/home/sourav/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
generator = EmbeddingGenerator()

# Load the data from the JSON file
with open('chunks_with_embeddings.json', 'r') as f:
    data = json.load(f)

# Extract chunks and their embeddings
chunks = [item['chunk_text'] for item in data]
chunk_embeddings = torch.tensor([item['embedding'] for item in data], dtype=torch.float32)


In [4]:
prompts = [
    "What are the theoretical limitations of embedding-based retrieval?",
    "How does the geometry of the embedding space affect retrieval performance?",
    "What is the 'norm concentration' phenomenon in high-dimensional spaces?",
    "Can embedding models truly capture semantic similarity for all tasks?",
    "What are the failure modes for embedding-based search?",
    "How can neural retrieval be optimized for hardware accelerators?",
    "What are the bottlenecks in modern neural retrieval pipelines?",
    "What is the trade-off between retrieval effectiveness and efficiency on accelerators?",
    "Explain the concept of query-side latency in neural retrieval.",
    "What are some state-of-the-art techniques for efficient neural retrieval on GPUs or TPUs?"
]

In [5]:
def retrieve_with_mixture_of_logits(query: str, top_k: int = 5, 
                                  temperature: float = 2.0) -> List[Tuple[float, str]]:
    """
    Retrieves the top_k most relevant chunks using a mixture of logits approach.
    
    Args:
        query: The query string
        top_k: Number of top results to return
        temperature: Temperature parameter for softmax scaling
    
    Returns:
        List of (score, chunk) tuples
    """
    # Get query embedding using our generator
    query_embedding = torch.tensor(generator.get_embedding(query))
    # Compute raw logits (dot product)
    logits = torch.matmul(chunk_embeddings, query_embedding)
    
    # Apply temperature scaling
    scaled_logits = logits / temperature
    
    # Apply softmax to get attention weights
    attention_weights = torch.nn.functional.softmax(scaled_logits, dim=0)
    
    # Get top-k indices and scores
    top_k_values, top_k_indices = torch.topk(attention_weights, k=top_k)
    
    # Create result list
    results = [(score.item(), chunks[idx.item()]) 
              for score, idx in zip(top_k_values, top_k_indices)]
    
    return results
    

In [6]:
# Function to print results
def print_results(query: str, results: List[Tuple[float, str]]):
    print(f"\n=== Query: {query} ===")
    for score, chunk in results:
        print(f"\nScore: {score:.4f}")
        print(f"Chunk: {chunk}")
    print("-" * 80)

# Run retrieval for each prompt
for prompt in prompts:
    results = retrieve_with_mixture_of_logits(prompt)
    print_results(prompt, results)


=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Score: 0.0119
Chunk: On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Iftekhar Naim1 and Jinhyuk Lee1
1Google DeepMind,2Johns Hopkins University
Vector embeddings have been tasked with an ever-increasing set of retrieval tasks over the years, with a
nascentriseinusingthemforreasoning,instruction-following,coding, andmore. Thesenewbenchmarks
push embeddings to work forany queryand any notion of relevancethat could be given. While prior
works have pointed out theoretical limitations of vector embeddings, there is a common assumption
that these difficulties are exclusively due to unrealistic queries, and those that are not can be overcome
with better training data and larger models. In this work, we demonstrate that we may encounter these
theoretical limitations in realistic settings with extremely simple queries. We connect known results
in learning theory, 

In [7]:
def retrieve_with_cosine(query: str, top_k: int = 5) -> List[Tuple[float, str]]:
    """
    Simple cosine similarity retrieval for comparison
    """
    query_embedding = torch.tensor(generator.get_embedding(query))
    
    # Compute cosine similarity
    similarities = torch.nn.functional.cosine_similarity(
        chunk_embeddings, query_embedding.unsqueeze(0))
    
    # Get top-k results
    top_k_values, top_k_indices = torch.topk(similarities, k=top_k)
    
    return [(score.item(), chunks[idx.item()]) 
            for score, idx in zip(top_k_values, top_k_indices)]

In [8]:
# Compare both methods
prompt = prompts[0]  # Take first prompt as example
print("\n=== Comparing retrieval methods ===")
print("\nMixture of Logits results:")
mol_results = retrieve_with_mixture_of_logits(prompt)
print_results(prompt, mol_results)

print("\nCosine Similarity results:")
cos_results = retrieve_with_cosine(prompt)
print_results(prompt, cos_results)


=== Comparing retrieval methods ===

Mixture of Logits results:

=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Score: 0.0119
Chunk: On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Iftekhar Naim1 and Jinhyuk Lee1
1Google DeepMind,2Johns Hopkins University
Vector embeddings have been tasked with an ever-increasing set of retrieval tasks over the years, with a
nascentriseinusingthemforreasoning,instruction-following,coding, andmore. Thesenewbenchmarks
push embeddings to work forany queryand any notion of relevancethat could be given. While prior
works have pointed out theoretical limitations of vector embeddings, there is a common assumption
that these difficulties are exclusively due to unrealistic queries, and those that are not can be overcome
with better training data and larger models. In this work, we demonstrate that we may encounter these
theoretical limitations in realistic settings with extrem

In [9]:
def compare_methods_detailed(prompt: str, top_k: int = 5):
    # Get results from both methods
    mol_results = retrieve_with_mixture_of_logits(prompt, top_k)
    cos_results = retrieve_with_cosine(prompt, top_k)
    
    print(f"\n=== Query: {prompt} ===")
    print("\nMixture of Logits vs Cosine Similarity:")
    print(f"{'Index':<6} {'MoL Score':^12} {'Cos Score':^12} {'Same?':^8}")
    print("-" * 40)
    
    # Compare each position
    for i in range(top_k):
        mol_score, mol_chunk = mol_results[i]
        cos_score, cos_chunk = cos_results[i]
        is_same = mol_chunk == cos_chunk
        print(f"{i:<6} {mol_score:^12.4f} {cos_score:^12.4f} {str(is_same):^8}")
    
    # Show sum of probabilities for MoL
    mol_sum = sum(score for score, _ in mol_results)
    print(f"\nSum of top-{top_k} MoL scores: {mol_sum:.4f}")

# Test with the first prompt
compare_methods_detailed(prompts[0])


=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Mixture of Logits vs Cosine Similarity:
Index   MoL Score    Cos Score    Same?  
----------------------------------------
0         0.0119       0.9233      True  
1         0.0117       0.8959      True  
2         0.0117       0.8921      True  
3         0.0117       0.8894      True  
4         0.0117       0.8893      True  

Sum of top-5 MoL scores: 0.0586


In [10]:
def test_temperature_effect(prompt: str, temperatures=[0.5, 1.0, 2.0, 4.0]):
    print(f"\n=== Testing different temperatures for: {prompt} ===")
    for temp in temperatures:
        results = retrieve_with_mixture_of_logits(prompt, temperature=temp)
        top_score = results[0][0]
        print(f"Temperature {temp:3.1f}: Top score = {top_score:.4f}")

test_temperature_effect(prompts[0])


=== Testing different temperatures for: What are the theoretical limitations of embedding-based retrieval? ===
Temperature 0.5: Top score = 0.0148
Temperature 1.0: Top score = 0.0128
Temperature 2.0: Top score = 0.0119
Temperature 4.0: Top score = 0.0114


In [11]:
def neural_reranking_retrieval(query: str, initial_k: int = 10, final_k: int = 5):
    """
    Enhanced retrieval with neural reranking
    """
    # Stage 1: Get initial candidates using cosine similarity
    initial_results = retrieve_with_cosine(query, top_k=initial_k)
    
    # Stage 2: Prepare candidate set for reranking
    candidate_chunks = [chunk for _, chunk in initial_results]
    candidate_indices = [chunks.index(chunk) for chunk in candidate_chunks]
    candidate_embeddings = chunk_embeddings[candidate_indices]
    
    # Stage 3: Cross-attention scoring
    query_embedding = torch.tensor(generator.get_embedding(query))
    
    # Compute attention scores between all pairs
    attention_matrix = torch.matmul(candidate_embeddings, candidate_embeddings.T)
    
    # Combine with query relevance
    query_scores = torch.matmul(candidate_embeddings, query_embedding)
    
    # Final scoring incorporating both document-document and query-document relations
    final_scores = torch.nn.functional.softmax(
        (attention_matrix.mean(dim=1) + query_scores) / 2.0, 
        dim=0
    )
    
    # Get top-k after reranking
    top_k_values, top_k_indices = torch.topk(final_scores, k=final_k)
    
    return [(score.item(), candidate_chunks[idx.item()]) 
            for score, idx in zip(top_k_values, top_k_indices)]

In [12]:
def compare_retrieval_methods(query: str):
    print("\n=== Simple Vector Search ===")
    simple_results = retrieve_with_cosine(query, top_k=5)
    print_results(query, simple_results)
    
    print("\n=== Neural Reranking ===")
    neural_results = neural_reranking_retrieval(query)
    print_results(query, neural_results)

# Test with first prompt
compare_retrieval_methods(prompts[0])


=== Simple Vector Search ===

=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Score: 0.9233
Chunk: On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Iftekhar Naim1 and Jinhyuk Lee1
1Google DeepMind,2Johns Hopkins University
Vector embeddings have been tasked with an ever-increasing set of retrieval tasks over the years, with a
nascentriseinusingthemforreasoning,instruction-following,coding, andmore. Thesenewbenchmarks
push embeddings to work forany queryand any notion of relevancethat could be given. While prior
works have pointed out theoretical limitations of vector embeddings, there is a common assumption
that these difficulties are exclusively due to unrealistic queries, and those that are not can be overcome
with better training data and larger models. In this work, we demonstrate that we may encounter these
theoretical limitations in realistic settings with extremely simple queries. We connect know

In [13]:
def analyze_retrieval_differences(prompts: List[str], top_k: int = 5):
    """
    Analyzes differences between cosine similarity and neural reranking for all prompts.
    """
    print(f"\n{'='*80}")
    print(f"Analyzing differences between retrieval methods for {len(prompts)} prompts")
    print(f"{'='*80}")
    
    for idx, prompt in enumerate(prompts, 1):
        # Get results from both methods
        cosine_results = retrieve_with_cosine(prompt, top_k=top_k)
        neural_results = neural_reranking_retrieval(prompt)
        
        # Extract chunks for easy comparison
        cosine_chunks = [chunk for _, chunk in cosine_results]
        neural_chunks = [chunk for _, chunk in neural_results]
        
        # Find differences
        different_chunks = set(cosine_chunks) ^ set(neural_chunks)
        different_order = cosine_chunks != neural_chunks
        
        print(f"\nPrompt {idx}: {prompt}")
        print("-" * 40)
        
        if different_chunks or different_order:
            print("⚠️ Differences detected!")
            
            if different_chunks:
                print("\nDifferent chunks retrieved:")
                print("Only in Cosine:", set(cosine_chunks) - set(neural_chunks))
                print("Only in Neural:", set(neural_chunks) - set(cosine_chunks))
            
            if different_order and not different_chunks:
                print("\nSame chunks but different ordering:")
                print("\nCosine order:")
                for i, (score, chunk) in enumerate(cosine_results, 1):
                    print(f"{i}. (Score: {score:.4f}) {chunk[:100]}...")
                
                print("\nNeural order:")
                for i, (score, chunk) in enumerate(neural_results, 1):
                    print(f"{i}. (Score: {score:.4f}) {chunk[:100]}...")
        else:
            print("✓ Both methods returned identical results in identical order")
        
        print(f"\n{'-'*80}")

# Run the analysis for all prompts
analyze_retrieval_differences(prompts)


Analyzing differences between retrieval methods for 10 prompts

Prompt 1: What are the theoretical limitations of embedding-based retrieval?
----------------------------------------
⚠️ Differences detected!

Same chunks but different ordering:

Cosine order:
1. (Score: 0.9233) On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Ift...
2. (Score: 0.8959) On the Theoretical Limitations of Embedding-Based Retrieval
Limitations
Although our experiments pro...
3. (Score: 0.8921) On the Theoretical Limitations of Embedding-Based Retrieval
5.6. Alternatives to Embedding Models
Ou...
4. (Score: 0.8894) setting where the vectors themselves are directly optimized with the test data. This allows us to
em...
5. (Score: 0.8893) On the Theoretical Limitations of Embedding-Based Retrieval
documents with logical operators), we wi...

Neural order:
1. (Score: 0.1026) On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Bora

In [14]:
def print_retrieval_statistics(prompts: List[str], top_k: int = 5):
    """
    Prints statistical summary of differences between retrieval methods.
    """
    total_prompts = len(prompts)
    different_chunks_count = 0
    different_order_count = 0
    
    for prompt in prompts:
        cosine_chunks = [chunk for _, chunk in retrieve_with_cosine(prompt, top_k=top_k)]
        neural_chunks = [chunk for _, chunk in neural_reranking_retrieval(prompt)]
        
        if set(cosine_chunks) != set(neural_chunks):
            different_chunks_count += 1
        elif cosine_chunks != neural_chunks:
            different_order_count += 1
    
    print("\nRetrieval Statistics:")
    print(f"Total prompts analyzed: {total_prompts}")
    print(f"Different chunks retrieved: {different_chunks_count} ({different_chunks_count/total_prompts*100:.1f}%)")
    print(f"Different ordering only: {different_order_count} ({different_order_count/total_prompts*100:.1f}%)")
    print(f"Identical results: {total_prompts-different_chunks_count-different_order_count} "
          f"({(total_prompts-different_chunks_count-different_order_count)/total_prompts*100:.1f}%)")

# Run both analyses
analyze_retrieval_differences(prompts)
print_retrieval_statistics(prompts)


Analyzing differences between retrieval methods for 10 prompts

Prompt 1: What are the theoretical limitations of embedding-based retrieval?
----------------------------------------
⚠️ Differences detected!

Same chunks but different ordering:

Cosine order:
1. (Score: 0.9233) On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Ift...
2. (Score: 0.8959) On the Theoretical Limitations of Embedding-Based Retrieval
Limitations
Although our experiments pro...
3. (Score: 0.8921) On the Theoretical Limitations of Embedding-Based Retrieval
5.6. Alternatives to Embedding Models
Ou...
4. (Score: 0.8894) setting where the vectors themselves are directly optimized with the test data. This allows us to
em...
5. (Score: 0.8893) On the Theoretical Limitations of Embedding-Based Retrieval
documents with logical operators), we wi...

Neural order:
1. (Score: 0.1026) On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Bora

In [15]:
def compare_reranking_effect(query: str, initial_k: int = 20, final_k: int = 5):
    """
    Demonstrates the effect of MOL reranking on vector DB results
    """
    # Step 1: Initial vector DB-style retrieval
    initial_results = retrieve_with_cosine(query, top_k=initial_k)
    
    # Step 2: Prepare candidates for MOL
    candidate_chunks = [chunk for _, chunk in initial_results]
    candidate_indices = [chunks.index(chunk) for chunk in candidate_chunks]
    candidate_embeddings = chunk_embeddings[candidate_indices]
    
    # Step 3: Apply MOL on just these candidates
    query_embedding = torch.tensor(generator.get_embedding(query))
    logits = torch.matmul(candidate_embeddings, query_embedding)
    attention_weights = torch.nn.functional.softmax(logits / 2.0, dim=0)
    
    # Get final results
    top_k_values, top_k_indices = torch.topk(attention_weights, k=final_k)
    mol_results = [(score.item(), candidate_chunks[idx.item()]) 
                  for score, idx in zip(top_k_values, top_k_indices)]
    
    # Print comparison
    print(f"\n=== Query: {query} ===")
    print("\nInitial Vector DB Results (top 5 of 20):")
    for i, (score, chunk) in enumerate(initial_results[:5], 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"Chunk: {chunk[:200]}...")
        
    print("\n" + "="*80)
    print("\nAfter MOL Reranking:")
    for i, (score, chunk) in enumerate(mol_results, 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"Chunk: {chunk[:200]}...")

# Test with first prompt
compare_reranking_effect(prompts[0])


=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Initial Vector DB Results (top 5 of 20):

1. Score: 0.9233
Chunk: On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Iftekhar Naim1 and Jinhyuk Lee1
1Google DeepMind,2Johns Hopkins University
Vector embeddings have been ...

2. Score: 0.8959
Chunk: On the Theoretical Limitations of Embedding-Based Retrieval
Limitations
Although our experiments provide theoretical insight for the most common type of embedding model
(single vector) they do not hol...

3. Score: 0.8921
Chunk: On the Theoretical Limitations of Embedding-Based Retrieval
5.6. Alternatives to Embedding Models
Our previous results show both theoretically and empirically that embedding models cannot represent
al...

4. Score: 0.8894
Chunk: setting where the vectors themselves are directly optimized with the test data. This allows us to
empirically show how the embedding dimension enables the solving 

In [16]:
def enhanced_mol_reranking(query: str, initial_k: int = 20, final_k: int = 5):
    """
    Enhanced MOL reranking that considers document-document relationships
    """
    # Step 1: Initial retrieval
    initial_results = retrieve_with_cosine(query, top_k=initial_k)
    candidate_chunks = [chunk for _, chunk in initial_results]
    candidate_indices = [chunks.index(chunk) for chunk in candidate_chunks]
    candidate_embeddings = chunk_embeddings[candidate_indices]
    
    # Step 2: Compute document-document similarities
    doc_doc_sim = torch.matmul(candidate_embeddings, candidate_embeddings.T)
    
    # Step 3: Query-document similarities
    query_embedding = torch.tensor(generator.get_embedding(query))
    query_doc_sim = torch.matmul(candidate_embeddings, query_embedding)
    
    # Step 4: Combine both signals
    # Factor in document uniqueness by penalizing similar documents
    diversity_penalty = 1.0 - (doc_doc_sim.mean(dim=1))  # Lower score for similar docs
    combined_scores = query_doc_sim + 0.3 * diversity_penalty
    
    # Step 5: Apply softmax for final scores
    final_scores = torch.nn.functional.softmax(combined_scores / 2.0, dim=0)
    
    # Get results
    top_k_values, top_k_indices = torch.topk(final_scores, k=final_k)
    mol_results = [(score.item(), candidate_chunks[idx.item()]) 
                  for score, idx in zip(top_k_values, top_k_indices)]
    
    return mol_results

# Compare original vs enhanced reranking
def compare_reranking_methods(query: str):
    print(f"\n=== Query: {query} ===")
    
    # Original cosine results
    print("\nOriginal Vector DB Results:")
    cosine_results = retrieve_with_cosine(query, top_k=5)
    for i, (score, chunk) in enumerate(cosine_results, 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"Chunk: {chunk[:200]}...")
    
    print("\n" + "="*80)
    
    # Enhanced MOL results
    print("\nEnhanced MOL Reranking:")
    mol_results = enhanced_mol_reranking(query)
    for i, (score, chunk) in enumerate(mol_results, 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"Chunk: {chunk[:200]}...")

# Test with first prompt
compare_reranking_methods(prompts[0])


=== Query: What are the theoretical limitations of embedding-based retrieval? ===

Original Vector DB Results:

1. Score: 0.9233
Chunk: On the Theoretical Limitations of
Embedding-Based Retrieval
Orion Weller*,1,2, Michael Boratko1, Iftekhar Naim1 and Jinhyuk Lee1
1Google DeepMind,2Johns Hopkins University
Vector embeddings have been ...

2. Score: 0.8959
Chunk: On the Theoretical Limitations of Embedding-Based Retrieval
Limitations
Although our experiments provide theoretical insight for the most common type of embedding model
(single vector) they do not hol...

3. Score: 0.8921
Chunk: On the Theoretical Limitations of Embedding-Based Retrieval
5.6. Alternatives to Embedding Models
Our previous results show both theoretically and empirically that embedding models cannot represent
al...

4. Score: 0.8894
Chunk: setting where the vectors themselves are directly optimized with the test data. This allows us to
empirically show how the embedding dimension enables the solving of retrieval 